In [1]:
!pip install transformers datasets accelerate nvidia-ml-py3

                                              0.0/215.3 kB ? eta -:--:--
     -                                        10.2/215.3 kB ? eta -:--:--
     -                                        10.2/215.3 kB ? eta -:--:--
     -----                                 30.7/215.3 kB 262.6 kB/s eta 0:00:01
     ------------                          71.7/215.3 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 215.3/215.3 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/172.3 MB ? eta -:--:--
                                              0.3/172.3 MB 5.2 MB/s eta 0:00:33
                                             1.1/172.3 MB 11.5 MB/s eta 0:00:15
                                             2.5/172.3 MB 17.8 MB/s eta 0:00:10
                                             4.4/172.3 MB 23.4 MB/s eta 0:00:08
     -                                      

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\adamd\anaconda3\envs\gpu\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\adamd\anaconda3\envs\gpu\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\adamd\anaconda3\envs\gpu\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\adamd\anaconda3\envs\gpu\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 94, in read
    self.__buf.write(data)
  File "C:\Users\adamd\anaconda3\envs\gpu\lib\tempfile.py", line 478, in func_wrapper
    return func(*args, **kwargs)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\adamd\anaconda3\envs\gpu\lib\s

In [1]:
import numpy as np
from datasets import Dataset


seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, (dataset_size)),
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [2]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [3]:
print_gpu_utilization()

GPU memory occupied: 753 MB.


In [6]:
!nvidia-smi

Sun Apr 16 15:21:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 L...  WDDM | 00000000:01:00.0  On |                  N/A |
| N/A   64C    P8               14W /  N/A|    618MiB /  6144MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
import pandas as pd

from googletrans import Translator


COVID_DATA_PATH = "data/preprocessed/world_news_test"

data = pd.read_json(COVID_DATA_PATH, orient="records", lines=True)

def translate_text(row: pd.DataFrame) -> str:
    translator = Translator()
    return translator.translate(row['text'], src='en', dest='pl').text

def translate_dataset(dataset) -> None:
    dataset['text'] = dataset.apply(translate_text, axis=1)
    return dataset

translate_dataset(data)

,text,label,hashtags,emojis,polarity,subjectivity,sentiment
0,Dom Aide Dint Homeysaid najgorsza osoba Turn C...,1,[],[],-0.009826,0.482797,negative
1,Flynn Hillary Clinton Big Woman Campus Embuse ...,0,[],[],0.098977,0.487655,positive
